<a href="https://colab.research.google.com/github/Miguel13579/Ciencias_Dados/blob/main/Trabalhofinal1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q streamlit faiss-cpu sentence-transformers transformers accelerate
!wget -q -nc https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!mv ngrok /usr/local/bin
#Instala dependências para NLP, RAG e interface.
#Baixa e instala o ngrok para expor a interface Streamlit na web.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.81.0
    Uninstalling openai-1.81.0:
      Successfully uninstalled openai-1.81.0


In [ ]:
!pip install numpy

In [ ]:
!pip install pandas openpyxl

In [ ]:
import pandas as pd
df = pd.read_excel("Perguntas_IA_Sistema_RAG.xlsx")
documentos = df["Pergunta"].dropna().tolist()
print(f"Carregados {len(documentos)} documentos para indexar.")
#Carrega o conteúdo da planilha como base documental.

Carregados 10 documentos para indexar.


In [ ]:
from transformers import pipeline
gerador = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", device_map="auto", max_new_tokens=300)

def buscar_similares(pergunta, k=3):
    embed = modelo_embedding.encode([pergunta], convert_to_numpy=True).astype("float32")
    dists, indices = index.search(embed, k)
    return [(textos[i], dists[0][j]) for j, i in enumerate(indices[0])]

def gerar_resposta_rag(pergunta, k=3):
    trechos = buscar_similares(pergunta, k)

    # Elimina duplicatas, mantém apenas os textos
    textos_unicos = list(dict.fromkeys([trecho[0] for trecho in trechos]))

    contexto = "\n".join(textos_unicos)
    prompt = f"""### Instrução:
Com base nos textos abaixo, responda à pergunta do usuário.

### Contexto:
{contexto}

### Pergunta:
{pergunta}

### Resposta:"""

    resposta = gerador(prompt)[0]["generated_text"].split("### Resposta:")[-1].strip()
    return resposta, textos_unicos

Device set to use cpu


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')  # leve e eficiente
embeddings = model.encode(documentos, convert_to_numpy=True)
print(f"Embeddings gerados: shape {embeddings.shape}")
#Gera embeddings vetoriais dos documentos.
#all-MiniLM-L6-v2 é leve e eficiente.

Embeddings gerados: shape (10, 384)


In [ ]:
import faiss

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)
print(f"Índice FAISS criado com {index.ntotal} vetores.")
#Cria um índice vetorial simples (sem compressão, flat).
#dim é a dimensão dos embeddings (normalmente 384 para MiniLM).

Índice FAISS criado com 10 vetores.


In [ ]:
def buscar_similares(query, k=3):
    emb_query = model.encode([query], convert_to_numpy=True)
    D, I = index.search(emb_query, k)
    return [(documentos[i], D[0][idx]) for idx, i in enumerate(I[0])]
#Retorna os k documentos mais similares a uma pergunta.

In [ ]:
pergunta = "O que é Inteligência Artificial e como ela funciona?"
resposta, trechos = gerar_resposta_rag(pergunta)

print("Pergunta:", pergunta)
print("Resposta:", resposta)
print("\nTrechos usados:")
for trecho in trechos:
    print("-", trecho)

Pergunta: O que é Inteligência Artificial e como ela funciona?
Resposta: Avaliar a performance de um modelo de IA é complicado, mas pode ser feito com o uso de métodos de avaliação. No entanto, existem muitos métodos que podem ser usados para avaliar a performance de um modelo de IA, incluindo:
- Métricas de perda
- Métricas de confian

Trechos usados:
- O que é Inteligência Artificial e como ela funciona?
- Como avaliar a performance de um modelo de IA?
- O que é um sistema RAG e como ele melhora a geração de respostas?


In [ ]:
import streamlit as st

st.title("Sistema RAG - Perguntas e Respostas")

pergunta = st.text_input("Digite sua pergunta:")

if pergunta:
    resposta = gerar_resposta_rag(pergunta)
    st.markdown("### Resposta:")
    st.write(resposta)

2025-05-31 00:12:29.706 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:29.707 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:29.709 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:29.710 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:29.712 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:29.713 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:29.719 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:29.720 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
import streamlit as st
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# ---------------------------
# Carregar modelo de embeddings e LLM
# ---------------------------
modelo_embedding = SentenceTransformer("all-MiniLM-L6-v2")  # Leve, ideal para Colab
gerador = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", device_map="auto", max_new_tokens=300)

# ---------------------------
# Simular base vetorial FAISS
# ---------------------------
# Suponha que esses são os textos indexados do seu .xlsx
textos = [
    "Inteligência Artificial é a capacidade de máquinas realizarem tarefas humanas.",
    "A IA funciona por meio de algoritmos que aprendem com dados.",
    "Redes neurais artificiais são inspiradas no cérebro humano."
]

# Gerar embeddings
embeddings = modelo_embedding.encode(textos, convert_to_numpy=True).astype("float32")
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

# ---------------------------
# Função de busca no índice + geração de resposta
# ---------------------------
def buscar_similares(pergunta, k=3):
    embed = modelo_embedding.encode([pergunta], convert_to_numpy=True).astype("float32")
    dists, indices = index.search(embed, k)
    return [textos[i] for i in indices[0]]
#Interface amigável onde o usuário insere perguntas.
#Usa st.write() para exibir a resposta e os trechos base.

def gerar_resposta_rag(pergunta, k=3):
    trechos = buscar_similares(pergunta, k)
    contexto = "\n".join(trechos)
    prompt = f"""### Instrução:
Com base nos textos abaixo, responda à pergunta do usuário.

### Contexto:
{contexto}

### Pergunta:
{pergunta}

### Resposta:"""

    resposta = gerador(prompt)[0]["generated_text"].split("### Resposta:")[-1].strip()
    return resposta, trechos

# ---------------------------
# Interface Streamlit
# ---------------------------
st.set_page_config(page_title="Sistema RAG", layout="centered")
st.title("🤖 Sistema RAG com TinyLlama + FAISS")

pergunta = st.text_input("Digite sua pergunta:")
if st.button("Responder") and pergunta:
    with st.spinner("Gerando resposta..."):
        resposta, trechos = gerar_resposta_rag(pergunta)
    st.subheader("📌 Resposta:")
    st.write(resposta)
    st.subheader("📚 Trechos usados:")
    for t in trechos:
        st.markdown(f"- {t}")

Device set to use cpu
2025-05-31 00:12:46.068 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:46.069 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:46.071 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:46.072 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:46.073 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:46.074 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:46.075 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-31 00:12:46.076 Thread 'MainThread': missing ScriptRunContext! This warning can be ignor

In [ ]:
!ngrok config add-authtoken SEU_TOKEN_NGROK
#Permite expor o app Streamlit fora do Colab.

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py
import streamlit as st
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline

modelo_embedding = SentenceTransformer("all-MiniLM-L6-v2")
gerador = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", device_map="auto", max_new_tokens=300)

textos = [
    "Inteligência Artificial é a capacidade de máquinas realizarem tarefas humanas.",
    "A IA funciona por meio de algoritmos que aprendem com dados.",
    "Redes neurais artificiais são inspiradas no cérebro humano."
]

embeddings = modelo_embedding.encode(textos, convert_to_numpy=True).astype("float32")
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

def buscar_similares(pergunta, k=3):
    embed = modelo_embedding.encode([pergunta], convert_to_numpy=True).astype("float32")
    dists, indices = index.search(embed, k)
    return [textos[i] for i in indices[0]]

def gerar_resposta_rag(pergunta, k=3):
    trechos = buscar_similares(pergunta, k)
    contexto = "\n".join(trechos)
    prompt = f"""### Instrução:
Com base nos textos abaixo, responda à pergunta do usuário.

### Contexto:
{contexto}

### Pergunta:
{pergunta}

### Resposta:"""

    resposta = gerador(prompt)[0]["generated_text"].split("### Resposta:")[-1].strip()
    return resposta, trechos

st.set_page_config(page_title="Sistema RAG", layout="centered")
st.title("🤖 Sistema RAG com TinyLlama + FAISS")

pergunta = st.text_input("Digite sua pergunta:")
if st.button("Responder") and pergunta:
    with st.spinner("Gerando resposta..."):
        resposta, trechos = gerar_resposta_rag(pergunta)
    st.subheader("📌 Resposta:")
    st.write(resposta)
    st.subheader("📚 Trechos usados:")
    for t in trechos:
        st.markdown(f"- {t}")
#Junta tudo em um único arquivo executável com streamlit run app.py.
#Útil para organização e reusabilidade.

Writing app.py
